In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
IMG_SIZE = 224
BATCH = 32
DATA_PATH = DATA_PATH = "C:\\Users\\DELL\\Desktop\\folder3\\train"
  # <-- ONLY ONE FOLDER (cats/dogs inside)

In [3]:
train_ds = image_dataset_from_directory(
    DATA_PATH,
    validation_split=0.2,   # 80% train
    subset="training",
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH
)

Found 1003 files belonging to 2 classes.
Using 803 files for training.


In [4]:
val_ds = image_dataset_from_directory(
    DATA_PATH,
    validation_split=0.2,   # 20% validation
    subset="validation",
    seed=42,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH
)

Found 1003 files belonging to 2 classes.
Using 200 files for validation.


In [5]:
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds  = val_ds.prefetch(tf.data.AUTOTUNE)

In [6]:
base_model = MobileNetV2(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

In [7]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.2)(x)
output = Dense(1, activation="sigmoid")(x)   # binary classification
model = Model(inputs=base_model.input, outputs=output)

model.compile(
    optimizer=Adam(1e-5),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 1,370,177 (5.23 MB)

 Non-trainable params: 1,051,904 (4.01 MB)

None


In [8]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100
)

Epoch 1/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 175s 3s/step - accuracy: 0.5965 - loss: 0.6778 - val_accuracy: 0.5150 - val_loss: 0.7021
Epoch 2/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.5903 - loss: 0.6528 - val_accuracy: 0.5450 - val_loss: 0.6951
Epoch 3/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 71s 3s/step - accuracy: 0.6413 - loss: 0.6439 - val_accuracy: 0.5350 - val_loss: 0.6863
Epoch 4/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 67s 3s/step - accuracy: 0.6526 - loss: 0.6192 - val_accuracy: 0.5500 - val_loss: 0.6797
Epoch 5/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 73s 3s/step - accuracy: 0.6924 - loss: 0.5731 - val_accuracy: 0.5800 - val_loss: 0.6736
Epoch 6/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 66s 3s/step - accuracy: 0.6849 - loss: 0.5693 - val_accuracy: 0.6000 - val_loss: 0.6682
Epoch 7/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 88s 3s/step - accuracy: 0.7347 - loss: 0.5460 - val_accuracy: 0.6300 - val_loss: 0.6625
Epoch 8/100
26/26 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.7397 - loss: 0.5214 - val_accuracy: 0.6550 - 

In [10]:
model.save("cats_dogs_transfer_learning.h5")
print("\nModel saved: cats_dogs_transfer_learning.h5")